# Santa 2025 - Quick EDA

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
getcontext().prec = 30

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head(10)

Shape: (20100, 4)
Columns: ['id', 'x', 'y', 'deg']


,id,x,y,deg
0,001_0,s0.0,s0.0,s90.0
1,002_0,s0.0,s0.0,s90.0
2,002_1,s0.202736,s-0.511271,s90.0
3,003_0,s0.0,s0.0,s90.0
4,003_1,s0.202736,s-0.511271,s90.0
5,003_2,s0.5206,s0.177413,s180.0
6,004_0,s0.0,s0.0,s90.0
7,004_1,s0.202736,s-0.511271,s90.0
8,004_2,s0.5206,s0.177413,s180.0
9,004_3,s-0.818657,s-0.228694,s180.0


In [2]:
# Parse the data
df['x'] = df['x'].astype(str).str.lstrip('s').astype(float)
df['y'] = df['y'].astype(str).str.lstrip('s').astype(float)
df['deg'] = df['deg'].astype(str).str.lstrip('s').astype(float)
df['n'] = df['id'].str.split('_').str[0].astype(int)
df['tree_idx'] = df['id'].str.split('_').str[1].astype(int)

print(f"N values: {df['n'].min()} to {df['n'].max()}")
print(f"Total rows: {len(df)}")
print(f"Expected rows: {sum(range(1, 201))} = 1+2+...+200")
print(f"Match: {len(df) == sum(range(1, 201))}")

N values: 1 to 200
Total rows: 20100
Expected rows: 20100 = 1+2+...+200
Match: True


In [3]:
# Load the best pre-optimized submission and calculate scores
best_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21156851249/submission/submission.csv')
best_df['x'] = best_df['x'].astype(str).str.lstrip('s').astype(float)
best_df['y'] = best_df['y'].astype(str).str.lstrip('s').astype(float)
best_df['deg'] = best_df['deg'].astype(str).str.lstrip('s').astype(float)
best_df['n'] = best_df['id'].str.split('_').str[0].astype(int)

# Calculate approximate side lengths (this is rough - actual needs polygon bounds)
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def get_tree_polygon(x, y, deg):
    base_poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(base_poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

def calculate_side(group):
    polys = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    union = unary_union(polys)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

# Calculate for a few N values
scores = []
for n in [1, 2, 3, 5, 10, 20, 50, 100, 150, 200]:
    group = best_df[best_df['n'] == n]
    side = calculate_side(group)
    score = side**2 / n
    scores.append({'n': n, 'side': side, 'score': score, 'efficiency': n / side**2})
    print(f"N={n:3d}: side={side:.6f}, score={score:.6f}, efficiency={n/side**2:.4f}")

print(f"\nTotal score (sampled): {sum(s['score'] for s in scores):.4f}")

N=  1: side=0.813173, score=0.661250, efficiency=1.5123
N=  2: side=0.949504, score=0.450779, efficiency=2.2184
N=  3: side=1.142031, score=0.434745, efficiency=2.3002
N=  5: side=1.443692, score=0.416850, efficiency=2.3989
N= 10: side=1.940696, score=0.376630, efficiency=2.6551
N= 20: side=2.742469, score=0.376057, efficiency=2.6592
N= 50: side=4.247076, score=0.360753, efficiency=2.7720
N=100: side=5.878188, score=0.345531, efficiency=2.8941
N=150: side=7.110529, score=0.337064, efficiency=2.9668
N=200: side=8.216619, score=0.337564, efficiency=2.9624

Total score (sampled): 4.0972


In [4]:
# Calculate full score
all_scores = []
for n in range(1, 201):
    group = best_df[best_df['n'] == n]
    side = calculate_side(group)
    score = side**2 / n
    all_scores.append({'n': n, 'side': side, 'score': score, 'efficiency': n / side**2})

total_score = sum(s['score'] for s in all_scores)
print(f"Total score: {total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {total_score - 68.919154:.6f}")

# Find N values with worst efficiency
import pandas as pd
scores_df = pd.DataFrame(all_scores)
print("\nTop 10 worst efficiency (most room for improvement):")
print(scores_df.nsmallest(10, 'efficiency')[['n', 'side', 'score', 'efficiency']])

Total score: 70.659437
Target: 68.919154
Gap: 1.740283

Top 10 worst efficiency (most room for improvement):
     n      side     score  efficiency
0    1  0.813173  0.661250    1.512287
1    2  0.949504  0.450779    2.218381
2    3  1.142031  0.434745    2.300198
4    5  1.443692  0.416850    2.398947
3    4  1.290806  0.416545    2.400702
6    7  1.673104  0.399897    2.500647
5    6  1.548438  0.399610    2.502438
8    9  1.867280  0.387415    2.581211
7    8  1.755921  0.385407    2.594658
14  15  2.384962  0.379203    2.637111


In [ ]:
# Score contribution by N range
print("Score contribution by N range:")
for start, end in [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]:
    range_scores = [s['score'] for s in all_scores if start <= s['n'] <= end]
    print(f"N={start:3d}-{end:3d}: {sum(range_scores):.4f} ({sum(range_scores)/total_score*100:.1f}%)")

print(f"\nTotal: {total_score:.6f}")